# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import gmaps
import os
import csv

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
csv_path = "../WeatherPy/city_results.cvs"

city_results_df = pd.read_csv(csv_path, index_col="City Name")

city_results_df.head()


,City Number,Country,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed
City Name,,,,,,,,
Albany,5106841,US,42.6001,-73.9662,12.99,50,78,3.47
Hilo,5855927,US,19.7297,-155.0900,82.40,51,20,5.75
Punta Arenas,3874787,CL,-53.1500,-70.9167,55.40,54,90,11.50
San Cristobal,3628473,VE,7.7669,-72.2250,66.87,74,53,1.07
Vila Franca do Campo,3372472,PT,37.7167,-25.4333,60.01,94,20,13.80


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#configure gmaps
gmaps.configure(api_key=g_key)

#latitude and longitude in locations
locations = city_results_df[["Latitude","Longitude"]]

#humidity as weight
rating = city_results_df["Humidity"]

#Plot Heatmap
new_york_coordinates = (40.75, -74.00)
fig = gmaps.figure(center=new_york_coordinates, zoom_level=2)

#create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=rating,
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)

#add layer
fig.add_layer(heat_layer)

#display
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
vacation_destinations = city_results_df
vacation_destinations


,City Number,Country,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed
City Name,,,,,,,,
Albany,5106841,US,42.6001,-73.9662,12.99,50,78,3.47
Hilo,5855927,US,19.7297,-155.0900,82.40,51,20,5.75
Punta Arenas,3874787,CL,-53.1500,-70.9167,55.40,54,90,11.50
San Cristobal,3628473,VE,7.7669,-72.2250,66.87,74,53,1.07
Vila Franca do Campo,3372472,PT,37.7167,-25.4333,60.01,94,20,13.80
...,...,...,...,...,...,...,...,...
São José da Coroa Grande,3388456,BR,-8.8978,-35.1478,78.80,81,11,10.42
Mariehamn,3041732,AX,60.0973,19.9348,30.20,74,0,13.80
Bogomila,792509,MK,41.5931,21.4717,23.00,63,0,14.97


In [5]:
drop_cities = vacation_destinations[vacation_destinations["Temperature"] > 85].index
vacation_destinations.drop(drop_cities, inplace=True)
vacation_destinations

,City Number,Country,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed
City Name,,,,,,,,
Albany,5106841,US,42.6001,-73.9662,12.99,50,78,3.47
Hilo,5855927,US,19.7297,-155.0900,82.40,51,20,5.75
Punta Arenas,3874787,CL,-53.1500,-70.9167,55.40,54,90,11.50
San Cristobal,3628473,VE,7.7669,-72.2250,66.87,74,53,1.07
Vila Franca do Campo,3372472,PT,37.7167,-25.4333,60.01,94,20,13.80
...,...,...,...,...,...,...,...,...
São José da Coroa Grande,3388456,BR,-8.8978,-35.1478,78.80,81,11,10.42
Mariehamn,3041732,AX,60.0973,19.9348,30.20,74,0,13.80
Bogomila,792509,MK,41.5931,21.4717,23.00,63,0,14.97


In [6]:
drop_cities = vacation_destinations[vacation_destinations["Temperature"] < 70].index
vacation_destinations.drop(drop_cities, inplace=True)
vacation_destinations

,City Number,Country,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed
City Name,,,,,,,,
Hilo,5855927,US,19.7297,-155.0900,82.40,51,20,5.75
Rikitea,4030556,PF,-23.1203,-134.9692,80.46,71,7,10.07
Honiara,2108502,SB,-9.4333,159.9500,81.95,81,100,7.83
Cidreira,3466165,BR,-30.1811,-50.2056,75.22,80,100,15.03
Bredasdorp,1015776,ZA,-34.5322,20.0403,72.00,78,0,3.00
...,...,...,...,...,...,...,...,...
Marsh Harbour,3571913,BS,26.5412,-77.0636,75.15,82,74,14.45
Tiarei,4033356,PF,-17.5333,-149.3333,80.60,74,75,23.02
São José da Coroa Grande,3388456,BR,-8.8978,-35.1478,78.80,81,11,10.42


In [7]:
drop_cities = vacation_destinations[vacation_destinations["Wind Speed"] > 10].index
vacation_destinations.drop(drop_cities, inplace=True)
vacation_destinations.head()

,City Number,Country,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed
City Name,,,,,,,,
Hilo,5855927,US,19.7297,-155.0900,82.40,51,20,5.75
Honiara,2108502,SB,-9.4333,159.9500,81.95,81,100,7.83
Bredasdorp,1015776,ZA,-34.5322,20.0403,72.00,78,0,3.00
Binga,895057,ZW,-17.6203,27.3414,71.58,89,100,6.38
Sémbé,2255304,CG,1.6481,14.5806,74.43,77,88,4.27


In [8]:
drop_cities = vacation_destinations[vacation_destinations["Cloudiness"] > 25].index
vacation_destinations.drop(drop_cities, inplace=True)
vacation_destinations.head(5)

,City Number,Country,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed
City Name,,,,,,,,
Hilo,5855927,US,19.7297,-155.0900,82.4,51,20,5.75
Bredasdorp,1015776,ZA,-34.5322,20.0403,72.0,78,0,3.00
George Town,1735106,MY,5.4112,100.3354,82.4,83,20,3.44
Hilo,5855927,US,19.7297,-155.0900,82.4,51,20,5.75
Bredasdorp,1015776,ZA,-34.5322,20.0403,72.0,78,0,3.00


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [16]:
vacation_destinations['Hotel Name'] = ""
vacation_destinations = vacation_destinations.head(10)
vacation_destinations

C:\Users\rache\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,City,City Number,Country,Lat,Lng,Temperature,Humidity,Cloudiness,Wind Speed,Hotel Name
0,Hilo,5855927,US,19.7297,-155.0900,82.40,51,20,5.75,
1,Bredasdorp,1015776,ZA,-34.5322,20.0403,72.00,78,0,3.00,
2,George Town,1735106,MY,5.4112,100.3354,82.40,83,20,3.44,
3,Hilo,5855927,US,19.7297,-155.0900,82.40,51,20,5.75,
4,Bredasdorp,1015776,ZA,-34.5322,20.0403,72.00,78,0,3.00,
5,George Town,1735106,MY,5.4112,100.3354,82.40,83,20,3.44,
6,Kapaa,5848280,US,22.0752,-159.3190,78.01,57,1,8.05,
7,Koungou,1090225,YT,-12.7336,45.2042,84.20,74,0,9.22,
8,Busselton,2075265,AU,-33.6500,115.3333,72.00,57,0,1.99,
9,Bonthe,2409914,SL,7.5264,-12.5050,77.45,87,0,4.03,


In [17]:
#Set parameters to search for hotels with 5000 meters
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {
              "rankby" : "distance",
              "type" : "lodging",
              "key" : g_key,
            }

In [11]:
for index, row in vacation_destinations.iterrows():
    #get lattitude and logitude from df
    lat = row["Latitude"]
    long = row["Longitude"]
    latlong = f'{lat},{long}'
    
    params["location"] = latlong
    
    response = requests.get(base_url, params=params).json()    
    
    print (response)
    results = response['results']
    
    #print(f'lat = {lat} long = {long}')
    try:
        vacation_destinations.loc[index,'Hotel Name'] = results[0]['name']
        
    except:
        print('--------')
        
vacation_destinations.reset_index()

{'html_attributions': [], 'next_page_token': 'ATtYBwKQzV96s3LfYtDHz3sE7PrhCOWdlPYEYDQpEybXLC9kdAlK0njlI2GTbatfJoWTrts91qPGg-4ZNHD6gzOTE_QmXqBtSphRqfzDBJGn6YUA8V7dgMGC9zAMf6xQrWtzRASTrfDLyA2ZbWI-lh8LaWq7lhRGUDACOn6qbwUkQrIDFYzAjmsrtI-Y_XXMk3RSBhMA1KUZE0qZJnRVIegWd8x47jZrs2FsMg7Mmax1xh1aUD3q0VqqCGzOhZAUoyZ4iMQQSobcpJsZSuha3DyHGV-me1upZ2vA4tjFMqIpw8PBNWAljcU3kM4h_7CIr8UZYbOk1eLXauoX1sFkfBJ7eHBWVe9MWxmS-bCZAD1a-zcB_sRHddTsmLO0xMAyfFLEseRe2po6VdahuGku-yBE4wjIRy_AgZL__8ZxqKSsrNnWn9RaU8eJyShJjQc', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 19.7292324, 'lng': -155.0904455}, 'viewport': {'northeast': {'lat': 19.7306887302915, 'lng': -155.0891173197085}, 'southwest': {'lat': 19.7279907697085, 'lng': -155.0918152802915}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'name': 'Dolphin Bay Hotel', 'photos': [{'height': 960, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/101444725316648766672">Dolphin

{'html_attributions': [], 'next_page_token': 'ATtYBwJQIflaHMGK3Nbd5Cwx8BbrX6uRui_Oxi1Ar92ToobRJyrK47-8S7BWqNgg9M4lB4ywapJ5sM6-BoFBpLZfqAeoEr7PoKcaYZktsymmz8lmK6tlQaa_PmJ5zRqCl4Y3U42l415uiBj98LO3hshPCJJ2qwZGUC7yB4jd0rhfw31kCmrCRLegjsQFewnIfjULRrg_2HnaJ5gsaeBUbull49VTgJu2tSUIooKJBUmjOIT4ozqn1dUxrfSqHmLNhs_HzNWUJFH0ZqX-Q28Ey3WB75r47pmfBq1IxZsnyU69yRuLfWSnN8IVdvW2LBJU-eSCwPycBGcJ3Fr9O44pywoLPy50dhC4jlObw6nFnDUavrnUS08KbL7H3XFKUzht1E64rh4eW_ExLR9fbpboeqDTY-h7_hXNfAYQdylL8OEEcTNIntDlVz3KUi7iwwU', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': -34.53186000000001, 'lng': 20.04055}, 'viewport': {'northeast': {'lat': -34.53045811970851, 'lng': 20.0418054802915}, 'southwest': {'lat': -34.5331560802915, 'lng': 20.0391075197085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'name': 'Firlane House', 'photos': [{'height': 1280, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/114497354307883028309">Tshiket

{'html_attributions': [], 'next_page_token': 'ATtYBwJ6AYooJlC3CecN8n5fAAgN8hWsoh8Jo3oAW17eWZ0qgU8BW_uUQNnzML-0xo5AZkp4g0l2Vi3eqxcbyTUZjLvL3qExeyqxq7fsoqU8kw9vWrUyJuGqgt2hDxCYeFKYp7pEKJ0l1-pIjx8edvsFDVlwWvL9szPfALecJpeMP90KDXL1lL5aIhp68I_BFoOYyvsuE9kx8CciyDSFITaPQBJj1z-lhYFCpwaVa2SkHMt9kJImk39kBYxxpqWX0qKpXtDJWrDBhYCPjSk7E72aF7FuWTFGRHBxDqo5_jwp71SR4G12XqksztRf-nQinP3ceF_wPBnzzP6MOb5JnvI4Pz5JGDChFRznAiuMQKtvf0ZOkkTGhOv-UE9kNZzreYfxE-9Z3FEfIyDaMvulR3BgMPCwL6d42nsUSyR1vG5se1SkODVv52N1xEmNbKI', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 5.4116716, 'lng': 100.3361075}, 'viewport': {'northeast': {'lat': 5.413058280291502, 'lng': 100.3374357802915}, 'southwest': {'lat': 5.410360319708498, 'lng': 100.3347378197085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'name': 'ZEN Rooms Victoria Street', 'place_id': 'ChIJL5ez-pLDSjARGgUQbbe3OUY', 'plus_code': {'compound_code': 'C86P+MC George Town, Penang, Malaysia', 'glob

{'html_attributions': [], 'next_page_token': 'ATtYBwKImTxhvCD-0Gj3IxKNdH4asbMMvdUNFjmo5h6R4EA-b0akzp34K9Z4SorW1cM9Ww7rR2ZPIJFRNQQXJ2KHV8rZJmHE6GwnTSBIreRUxMdEOnfOSAiIXBA3wl-0NfcPm8rVpk1dCWWOKdKa02NCcNS5XAk3AlG9RVTk9RugGEPzgmt75_bBwLb_heiBQnND6KaGRiUyqtFpWLp1cK2-idm9p52MuVOwIAMguj-d4stdHZzELFMlC1arp-i_NYXBSET8d9gctnUb8rnK3PTUYWmd3pkzvd6_2BsVYNjn2DniVCmtKMfp0D1QwKPmIeXkmEoxC-qbvPybLCgLGZpGAL1_r1WBzuGTtzAJ_2W5X0qvuWSRfHsGA7foF2MRmbPL-gzniQUErS5rObZYSRdHoKYI4t9TCRokKtRGA_GeDjt3xjHPu43BxyhDibA', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 19.7292324, 'lng': -155.0904455}, 'viewport': {'northeast': {'lat': 19.7306887302915, 'lng': -155.0891173197085}, 'southwest': {'lat': 19.7279907697085, 'lng': -155.0918152802915}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'name': 'Dolphin Bay Hotel', 'photos': [{'height': 960, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/101444725316648766672">Dolphin

{'html_attributions': [], 'next_page_token': 'ATtYBwJrcyKTyN4tb4X4rKVJdgfUI34Rd2l74jK1wKngJQZZeTXHDF1jUq4c2QOS219BPSADj1VgSslmo-VOTP20GTkGSU3ZpXiYxAnNxQj32MLGGZyG3gIrs2Rhh93LZUTJzrNWwysjqg0Jg7XrtiWLlCKvzNMjvWUhDAlknFtBZ2Q8e8HiYEb2rFXMf6NnRTK2e02fOOjELiJkZMjWS-hLkDvCanC8d39OjVVfOEulDYHk_XRNIQF-y7Zn3xKb_gtOlkHKY6XkmsY-9cu2GTz07tqQZNg3w059N29c9cXLNUEMRUz9TzaUbNXGMSzpBQdmztVkocHFzJTJYU5DuZAIOd9JJxWlS_YWvzfzCG_8JvsXoYZGsOvmSzD4M6JzGeBsjIVURsAPfus6--3lbKZ-3eqtqhJbDfO273vu-a8qVgHUOtTiRGObXGhnCRM', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': -34.53186000000001, 'lng': 20.04055}, 'viewport': {'northeast': {'lat': -34.53045811970851, 'lng': 20.0418054802915}, 'southwest': {'lat': -34.5331560802915, 'lng': 20.0391075197085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'name': 'Firlane House', 'photos': [{'height': 1280, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/114497354307883028309">Tshiket

...

{'html_attributions': [], 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 8.944933400000002, 'lng': 8.6017363}, 'viewport': {'northeast': {'lat': 8.946268580291502, 'lng': 8.603090480291502}, 'southwest': {'lat': 8.943570619708497, 'lng': 8.6003925197085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'name': 'Roma Zee Guest House Annex', 'photos': [{'height': 2560, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/111562622969316817365">Mohan Damar</a>'], 'photo_reference': 'ATtYBwIcUt2H8gevfsoEUBO6xfnqxWXcooIUNG2GAWxUte3P-5ZG4fnr2ATjYjC_Cx-xfVxgd8z8yLI5Pawam7CyBMYx3RLIhfelFUFAfA6yWBPnct3nnGa07AqPV2pl7QHEymuZ7bGrdgjQNe0NAPpz3EkDQbonfiAgldPiNE9IQOavmm_3', 'width': 1920}], 'place_id': 'ChIJU48gmkV6URARWnNxzP3_W1o', 'plus_code': {'compound_code': 'WJV2+XM Wamba, Nigeria', 'global_code': '6FWCWJV2+XM'}, 'rating': 4.2, 'reference': 'ChIJU48gmkV6URARWnNxzP3_W1o', 'scope': 'GOOGLE', 'types': ['lodgin

{'html_attributions': [], 'next_page_token': 'ATtYBwIZR4AttmexseO8tzL8ysQqheYOZzMWtkzwk3zHRwg9M2-AyaDmSnZTWDva1GTIxuYqiCBhuMlGnW4QTX-19PjcR3SrVHBh36i61y_L35jDN7xRz-Hy51G66G_Mn3uGDVkAQsAOcip5pTCFHDvRn1YPgvohca-qJ5aaEaRs6g9XpZel2t7ZoUhiE1ckW5B0mMRBI9eXheC6vdzsF0fG1_iYs-SpUMCU1Fwe_A-JNb7Fgcpf4xE85KuacmG82aup9QKPWWwmMvEIOox8xZ93Zq3LXLqzbxLQ1COmSk82evJ3ZZpeF25I4GfboI-ylQHJGl0wXPw2hkOJqtx0tqvAK_wOquP8pd82psZpJq0tn7yI4Q8em6DESDG63mIlk_vaafNfCWeNzFLiiAZL39OMdumj_3pxVlzlVEGq3VQvCbyWVZhy4O7aW-5VPd0', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': -15.1938478, 'lng': 12.1529643}, 'viewport': {'northeast': {'lat': -15.1925373697085, 'lng': 12.1543420802915}, 'southwest': {'lat': -15.1952353302915, 'lng': 12.1516441197085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'name': 'Restaurate Sol e Mar', 'opening_hours': {'open_now': False}, 'photos': [{'height': 2448, 'html_attributions': ['<a href="https://maps.google.com/maps

{'html_attributions': [], 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 17.5166234, 'lng': -101.2585348}, 'viewport': {'northeast': {'lat': 17.5179872802915, 'lng': -101.2571187197085}, 'southwest': {'lat': 17.5152893197085, 'lng': -101.2598166802915}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'name': 'Motel Casa Blanca', 'opening_hours': {'open_now': True}, 'place_id': 'ChIJw3jQm7aINIQRpR5DBJQAA1s', 'plus_code': {'compound_code': 'GP8R+JH Petatlán, Guerrero, Mexico', 'global_code': '759WGP8R+JH'}, 'rating': 3.7, 'reference': 'ChIJw3jQm7aINIQRpR5DBJQAA1s', 'scope': 'GOOGLE', 'types': ['lodging', 'point_of_interest', 'establishment'], 'user_ratings_total': 10, 'vicinity': 'Petatlán'}, {'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 17.53333749999999, 'lng': -101.2630163}, 'viewport': {'northeast': {'lat': 17.5345984302915, 'lng': -101.2617436697085}, 'southwest': {'lat': 17.5319004697085,

{'html_attributions': [], 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 9.1311842, 'lng': 18.3921482}, 'viewport': {'northeast': {'lat': 9.1324707802915, 'lng': 18.3935326802915}, 'southwest': {'lat': 9.129772819708496, 'lng': 18.3908347197085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'name': 'Auberge Ndoumbelane', 'place_id': 'ChIJa8SJ5nk_2RAR5CAMW_lArdM', 'plus_code': {'compound_code': '49JR+FV Sarh, Chad', 'global_code': '6FXW49JR+FV'}, 'reference': 'ChIJa8SJ5nk_2RAR5CAMW_lArdM', 'scope': 'GOOGLE', 'types': ['lodging', 'point_of_interest', 'establishment'], 'vicinity': 'Sarh'}, {'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 9.131731300000002, 'lng': 18.39502809999999}, 'viewport': {'northeast': {'lat': 9.133060730291504, 'lng': 18.3963397302915}, 'southwest': {'lat': 9.1303627697085, 'lng': 18.3936417697085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lo

,City Name,City Number,Country,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Hotel Name
0,Hilo,5855927,US,19.7297,-155.0900,82.40,51,20,5.75,Dolphin Bay Hotel
1,Bredasdorp,1015776,ZA,-34.5322,20.0403,72.00,78,0,3.00,Firlane House
2,George Town,1735106,MY,5.4112,100.3354,82.40,83,20,3.44,ZEN Rooms Victoria Street
3,Hilo,5855927,US,19.7297,-155.0900,82.40,51,20,5.75,Dolphin Bay Hotel
4,Bredasdorp,1015776,ZA,-34.5322,20.0403,72.00,78,0,3.00,Firlane House
5,George Town,1735106,MY,5.4112,100.3354,82.40,83,20,3.44,ZEN Rooms Victoria Street
6,Kapaa,5848280,US,22.0752,-159.3190,78.01,57,1,8.05,Pono Kai Resort
7,Koungou,1090225,YT,-12.7336,45.2042,84.20,74,0,9.22,Villa Netibor
8,Busselton,2075265,AU,-33.6500,115.3333,72.00,57,0,1.99,Pearls Place
9,Bonthe,2409914,SL,7.5264,-12.5050,77.45,87,0,4.03,Bonthe Holiday Village


In [12]:
hotel_df = vacation_destinations
hotel_df.reset_index(inplace=True)
hotel_df.rename(columns={'Latitude':'Lat',
                         'Longitude' : 'Lng',
                         'City Name' : 'City'
                        }, inplace=True)
hotel_df

,City,City Number,Country,Lat,Lng,Temperature,Humidity,Cloudiness,Wind Speed,Hotel Name
0,Hilo,5855927,US,19.7297,-155.0900,82.40,51,20,5.75,Dolphin Bay Hotel
1,Bredasdorp,1015776,ZA,-34.5322,20.0403,72.00,78,0,3.00,Firlane House
2,George Town,1735106,MY,5.4112,100.3354,82.40,83,20,3.44,ZEN Rooms Victoria Street
3,Hilo,5855927,US,19.7297,-155.0900,82.40,51,20,5.75,Dolphin Bay Hotel
4,Bredasdorp,1015776,ZA,-34.5322,20.0403,72.00,78,0,3.00,Firlane House
5,George Town,1735106,MY,5.4112,100.3354,82.40,83,20,3.44,ZEN Rooms Victoria Street
6,Kapaa,5848280,US,22.0752,-159.3190,78.01,57,1,8.05,Pono Kai Resort
7,Koungou,1090225,YT,-12.7336,45.2042,84.20,74,0,9.22,Villa Netibor
8,Busselton,2075265,AU,-33.6500,115.3333,72.00,57,0,1.99,Pearls Place
9,Bonthe,2409914,SL,7.5264,-12.5050,77.45,87,0,4.03,Bonthe Holiday Village


In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
##hotel_info = [info_box_template.format(**row) for index, row in vacation_destinations.iterrows()]
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]
#locations = vacation_destinations[["Latitude", "Longitude"]]
locations.head(10)

,Lat,Lng
0,19.7297,-155.0900
1,-34.5322,20.0403
2,5.4112,100.3354
3,19.7297,-155.0900
4,-34.5322,20.0403
5,5.4112,100.3354
6,22.0752,-159.3190
7,-12.7336,45.2042
8,-33.6500,115.3333
9,7.5264,-12.5050


In [14]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)

fig.add_layer(markers)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))